# COVID-19: Global EDA and TS Prediction For Pakistan using SIRD Model

### Importing Libraries

In [ ]:
import numpy as np
import pandas as pd
from datetime import datetime
import matplotlib.pyplot as plt
from sklearn import preprocessing
from scipy.integrate import odeint
from scipy import integrate, optimize
import plotly.express as px
import plotly.io as pio
pio.templates.default = "plotly_dark"
import matplotlib.pyplot as plt
%matplotlib inline 

import warnings
warnings.filterwarnings('ignore')

### Data Loading and Cleaning 

Import Population Data



In [ ]:
pop = pd.read_csv('../input/population-by-country-2020/population_by_country_2020.csv')
pop.head()

In [ ]:
pop.rename(columns={'Country (or dependency)':'country',
                   'Population (2020)':'population'
                   }, inplace=True)

### Worldwide cleaned data

In [ ]:
cleaned_data = pd.read_csv('https://raw.githubusercontent.com/imdevskp/covid_19_jhu_data_web_scrap_and_cleaning/master/covid_19_clean_complete.csv')
cleaned_data.head()

The cleaned data from [COVID-19 Complete Dataset (Updated every 24hrs)](https://www.kaggle.com/imdevskp/corona-virus-report) is used for visualizations. The file contains the cumulative count of confirmed, death and recovered cases of COVID-19 from different countries from 22nd January 2020.

In [ ]:
cleaned_data.rename(columns={'Date': 'date', 
                     'Province/State':'state',
                     'Country/Region':'country',
                     'Confirmed': 'confirmed',
                     'Deaths':'deaths',
                     'Recovered':'recovered'
                    }, inplace=True)

# cases 
cases = ['confirmed', 'deaths', 'recovered', 'active']

# Active Case = confirmed - deaths - recovered
cleaned_data['active'] = cleaned_data['confirmed'] - cleaned_data['deaths'] - cleaned_data['recovered']

# replacing Mainland china with just China
cleaned_data['country'] = cleaned_data['country'].replace('Mainland China', 'China')

# filling missing values 
cleaned_data[['state']] = cleaned_data[['state']].fillna('')
cleaned_data[cases] = cleaned_data[cases].fillna(0)
data = cleaned_data.copy()

In [ ]:
data['date'] =pd.to_datetime(data['date'], infer_datetime_format=True)

In [ ]:
print("External Data")
print(f"Earliest Entry: {data['date'].min()}")
print(f"Last Entry:     {data['date'].max()}")
print(f"Total Days:     {data['date'].max() - data['date'].min()}")

### USA county wise data

In [ ]:
us_data = pd.read_csv("https://raw.githubusercontent.com/imdevskp/covid_19_jhu_data_web_scrap_and_cleaning/master/usa_county_wise.csv")

In [ ]:
us_data.columns

In [ ]:
#Cleaning
us_data.drop(['UID', 'iso2', 'iso3', 'code3', 'FIPS', 'Admin2','Lat','Long_', 'Combined_Key'], inplace=True, axis=1)

us_data = us_data[~us_data.Province_State.str.match("Diamond Princess|Grand Princess|Recovered|Northern Mariana Islands|American Samoa")]

us_data.rename(columns={'Date': 'date', 
                     'Province_State':'state',
                     'Country_Region':'country',
                     'Confirmed': 'confirmed',
                     'Deaths': 'deaths'
                    }, inplace=True)

us_data['date'] =pd.to_datetime(us_data['date'], infer_datetime_format=True)

In [ ]:
us_data.head()

## Data Analysis - COVID-19

### 1. Confirmed Cases Over Time

Now lets take a look at the confirmed cases across the globe.

In [ ]:
ww_df = data.groupby('date')[['confirmed', 'deaths']].sum().reset_index()
ww_df['new_case'] = ww_df['confirmed'] - ww_df['confirmed'].shift(1)
ww_df.tail()

In [ ]:
fig = px.line(ww_df, x="date", y="confirmed", title="Worldwide Confirmed Cases Over Time")
fig.show()

#### Country-wise growth

What kind of country is in the dataset? How's the distribution of number of confirmed cases by country?

In [ ]:
countries = data['country'].unique()
print(f'{len(countries)} countries are in dataset:\n{countries}')

In [ ]:
target_date = data['date'].max()

print('Date: ', target_date)
for i in [1, 10, 100, 1000, 10000]:
    n_countries = len(data.query('(date == @target_date) & confirmed > @i'))
    print(f'{n_countries} countries have more than {i} confirmed cases')

It is difficult to see all countries so let's check countries of our interest.

In [ ]:
grouped_china = data[data['country'] == "China"].reset_index()
grouped_china_date = grouped_china.groupby('date')['date', 'confirmed', 'deaths', 'active', 'recovered'].sum().reset_index()

grouped_italy = data[data['country'] == "Italy"].reset_index()
grouped_italy_date = grouped_italy.groupby('date')['date', 'confirmed', 'deaths', 'active', 'recovered'].sum().reset_index()

grouped_us = data[data['country'] == "US"].reset_index()
grouped_us_date = grouped_us.groupby('date')['date', 'confirmed', 'deaths', 'active', 'recovered'].sum().reset_index()

grouped_spain = data[data['country'] == "Spain"].reset_index()
grouped_spain_date = grouped_spain.groupby('date')['date', 'confirmed', 'deaths', 'active', 'recovered'].sum().reset_index()

grouped_pak = data[data['country'] == "Pakistan"].reset_index()
grouped_pak_date = grouped_pak.groupby('date')['date', 'confirmed', 'deaths', 'active', 'recovered'].sum().reset_index()

grouped_ind = data[data['country'] == "India"].reset_index()
grouped_ind_date = grouped_ind.groupby('date')['date', 'confirmed', 'deaths', 'active', 'recovered'].sum().reset_index()

grouped_rest = data[~data['country'].isin(['China', 'Italy', 'US', 'Spain', 'Pakistan', 'India'])].reset_index()
grouped_rest_date = grouped_rest.groupby('date')['date', 'confirmed', 'deaths', 'active', 'recovered'].sum().reset_index()

In [ ]:
plot_titles = ['China', 'Spain', 'Italy', 'USA', 'Pakistan', 'India', 'Rest of the World']


fig = px.line(grouped_china_date, x="date", y="confirmed", 
              title=f"Confirmed Cases in {plot_titles[0].upper()} Over Time", 
              color_discrete_sequence=['#F61067'],
              height=500
             )
fig.show()

fig = px.line(grouped_spain_date, x="date", y="confirmed", 
              title=f"Confirmed Cases in {plot_titles[1].upper()} Over Time", 
              color_discrete_sequence=['#F61407'],
              height=500
             )
fig.show()

fig = px.line(grouped_italy_date, x="date", y="confirmed", 
              title=f"Confirmed Cases in {plot_titles[2].upper()} Over Time", 
              color_discrete_sequence=['#91C4F2'],
              height=500
             )
fig.show()

fig = px.line(grouped_us_date, x="date", y="confirmed", 
              title=f"Confirmed Cases in {plot_titles[3].upper()} Over Time", 
              color_discrete_sequence=['#6F2DBD'],
              height=500
             )
fig.show()

fig = px.line(grouped_pak_date, x="date", y="confirmed", 
              title=f"Confirmed Cases in {plot_titles[4].upper()} Over Time", 
              color_discrete_sequence=['#FFDF64'],
              height=500
             )
fig.show()

fig = px.line(grouped_ind_date, x="date", y="confirmed", 
              title=f"Confirmed Cases in {plot_titles[5].upper()} Over Time", 
              color_discrete_sequence=['#FFDF64'],
              height=500
             )
fig.show()

fig = px.line(grouped_rest_date, x="date", y="confirmed", 
              title=f"Confirmed Cases in {plot_titles[6].upper()} Over Time", 
              color_discrete_sequence=['#FFDF64'],
              height=500
             )
fig.show()

In [ ]:
data['state'] = data['state'].fillna('')
temp = data[[col for col in data.columns if col != 'state']]

latest = temp[temp['date'] == max(temp['date'])].reset_index()
latest_grouped = latest.groupby('country')['confirmed', 'deaths'].sum().reset_index()

In [ ]:
fig = px.choropleth(latest_grouped, locations="country", 
                    locationmode='country names', color="confirmed", 
                    hover_name="country", range_color=[1,5000], 
                    color_continuous_scale="peach", 
                    title='Countries with Confirmed Cases')
fig.show()

> #### Feel free to zoom into the interactive maps.

The above graph is just an illustration of how the virus is spread out across the globe.



In [ ]:
fig = px.bar(latest_grouped.sort_values('confirmed', ascending=False)[:20][::-1], 
             x='confirmed', y='country',
             title='Confirmed Cases Worldwide', text='confirmed', height=1000, orientation='h')
fig.show()

Looking at the numbers it's worth noting that Europe is more affected than China and it's neighbors. 

Currently, the US being the most affected country.

In [ ]:
usa_latest = us_data[us_data['date'] == max(us_data['date'])]
usa_latest = usa_latest.groupby('state')['confirmed', 'deaths'].max().reset_index()

fig = px.bar(usa_latest.sort_values('confirmed', ascending=False)[:10][::-1], 
             x='confirmed', y='state', color_discrete_sequence=['#D63230'],
             title='Confirmed Cases in USA', text='confirmed', orientation='h')
fig.show()

Looking at the American states, New York being the clear and outright most affected state.

## 2. Confirmed Deaths Over Time

In [ ]:
#ww_death_df = data.groupby('date')['deaths'].sum().reset_index()
fig = px.line(ww_df, x="date", y="deaths", title="Worldwide Deaths Over Time",
             color_discrete_sequence=['#F42272'])
fig.show()

In [ ]:
ww_df['mortality'] = ww_df['deaths'] / ww_df['confirmed']

fig = px.line(ww_df, x="date", y="mortality", 
              title="Worldwide Mortality Rate Over Time")
fig.show()

We see that mortality rate is kept almost 3%, however it is slightly increasing gradually to go over 7% at the end of April.

Global Death tolls have began to rise sharply ever since the turn of March.

In [ ]:
plot_titles = ['China', 'Spain', 'Italy', 'USA', 'Pakistan', 'India', 'Rest of the World']

fig = px.line(grouped_china_date, x="date", y="deaths", 
              title=f"Deaths in {plot_titles[0].upper()} Over Time", 
              color_discrete_sequence=['#F61067'],
              height=500
             )
fig.show()

fig = px.line(grouped_spain_date, x="date", y="deaths", 
              title=f"Deaths in {plot_titles[1].upper()} Over Time", 
              color_discrete_sequence=['#F61067'],
              height=500
             )
fig.show()

fig = px.line(grouped_italy_date, x="date", y="deaths", 
              title=f"Deaths in {plot_titles[2].upper()} Over Time", 
              color_discrete_sequence=['#91C4F2'],
              height=500
             )
fig.show()

fig = px.line(grouped_us_date, x="date", y="deaths", 
              title=f"Deaths in {plot_titles[3].upper()} Over Time", 
              color_discrete_sequence=['#6F2DBD'],
              height=500
             )
fig.show()

fig = px.line(grouped_pak_date, x="date", y="deaths", 
              title=f"Deaths in {plot_titles[4].upper()} Over Time", 
              color_discrete_sequence=['#FFDF64'],
              height=500
             )
fig.show()

fig = px.line(grouped_ind_date, x="date", y="deaths", 
              title=f"Deaths in {plot_titles[5].upper()} Over Time", 
              color_discrete_sequence=['#FFDF64'],
              height=500
             )
fig.show()

fig = px.line(grouped_rest_date, x="date", y="deaths", 
              title=f"Deaths in {plot_titles[6].upper()} Over Time", 
              color_discrete_sequence=['#FFDF64'],
              height=500
             )
fig.show()

But deaths in China somehow seem to be decreased since April which is interesting.

Looking at the Deaths elsewhere is a saddening graph in itself. Terrible! 

Look at the graph of USA it is almost as if the curve is going into a straight like to the skies above. 

In [ ]:
fig = px.choropleth(latest_grouped, locations="country", 
                    locationmode='country names', color="deaths", 
                    hover_name="deaths", range_color=[1,100], 
                    color_continuous_scale="peach", 
                    title='Countries with Reported Deaths')
fig.show()

In [ ]:
fig = px.bar(latest_grouped.sort_values('deaths', ascending=False)[:10][::-1], 
             x='deaths', y='country',
             title='Confirmed Deaths Worldwide', text='deaths', orientation='h')
fig.show()

US has topped the list!

In [ ]:
fig = px.bar(usa_latest.sort_values('deaths', ascending=False)[:5][::-1], 
             x='deaths', y='state', color_discrete_sequence=['#D63230'],
             title='Deaths in USA', text='deaths', orientation='h')
fig.show()

Not a good time to be a New Yorkian I guess. 
Alarming signs for the US?

## 3. Active Cases Over Time

Now lets analyze an important thing - ACTIVE CASES over time. It is the number of people who are affected by the virus excluding the dead and the recovered. This might have a bigger say on what is to come.

In [ ]:
plot_titles = ['China', 'Spain', 'Italy', 'USA', 'Pakistan', 'India', 'Rest of the World']

fig = px.line(grouped_china_date, x="date", y="active", 
              title=f"Active Cases in {plot_titles[0].upper()} Over Time", 
              color_discrete_sequence=['#F61067'],
              height=500
             )
fig.show()

fig = px.line(grouped_spain_date, x="date", y="active", 
              title=f"Active Cases in {plot_titles[1].upper()} Over Time", 
              color_discrete_sequence=['#F61067'],
              height=500
             )
fig.show()

fig = px.line(grouped_italy_date, x="date", y="active", 
              title=f"Active Cases in {plot_titles[2].upper()} Over Time", 
              color_discrete_sequence=['#91C4F2'],
              height=500
             )
fig.show()

fig = px.line(grouped_us_date, x="date", y="active", 
              title=f"Active Cases in {plot_titles[3].upper()} Over Time", 
              color_discrete_sequence=['#6F2DBD'],
              height=500
             )
fig.show()

fig = px.line(grouped_pak_date, x="date", y="active", 
              title=f"Active Cases in {plot_titles[4].upper()} Over Time", 
              color_discrete_sequence=['#6F2DBD'],
              height=500
             )
fig.show()

fig = px.line(grouped_ind_date, x="date", y="active", 
              title=f"Active Cases in {plot_titles[5].upper()} Over Time", 
              color_discrete_sequence=['#6F2DBD'],
              height=500
             )
fig.show()

fig = px.line(grouped_rest_date, x="date", y="active", 
              title=f"Active Cases in {plot_titles[6].upper()} Over Time", 
              color_discrete_sequence=['#FFDF64'],
              height=500
             )
fig.show()

* Active cases in China have plummeted multifold over the past few days making it a much better sign for the country and its people. Something is working out there. Maybe the lockdown and the strict rules implied? Anyway there's a lot to learn from China this time.

* Active cases elsewhere have skyrocketed to be frank, which is deeply worrying. Especially USA again sporting an almost straight line graph to the skies. 

In [ ]:
cleaned_data['state'] = cleaned_data['state'].fillna('')
temp = cleaned_data[[col for col in cleaned_data.columns if col != 'state']]

latest = temp[temp['date'] == max(temp['date'])].reset_index()
latest_grouped = latest.groupby('country')['confirmed', 'deaths', 'active', 'recovered'].sum().reset_index()

In [ ]:
fig = px.choropleth(latest_grouped, locations="country", 
                    locationmode='country names', color="active", 
                    hover_name="active", range_color=[1,1000], 
                    color_continuous_scale="peach", 
                    title='Active Cases Worldwide')
fig.show()

* China seems quite safe now

* An interesting find from all these plots is that sub-saharan Africa seem to have fewer cases than the rest of the world. 

* However things might get a lot worse when it hits Africa, it is better to keep it as far away as possible. 

In [ ]:
fig = px.bar(latest_grouped.sort_values('active', ascending=False)[:10][::-1], 
             x='active', y='country',
             title='Active Cases Worldwide', text='active', orientation='h')
fig.show()

* It is very much possible to see things escalate in these few countries here even more with the rise in infected people.

* Much of what is to come over the next few days depend on what is going to happen here 


## 4. Recovered Cases 

In [ ]:
fig = px.bar(latest_grouped.sort_values('recovered', ascending=False)[:10][::-1], 
             x='recovered', y='country',
             title='Recovered Cases Worldwide', text='recovered', orientation='h')
fig.show()

* Wow this graph gives me some hope at last.

## 5. Mortality and Recovery Rates

It is worth seeing these stats as well. It might have a story for sure.

In [ ]:
cleaned_latest = cleaned_data[cleaned_data['date'] == max(cleaned_data['date'])]
flg = cleaned_latest.groupby('country')['confirmed', 'deaths', 'recovered'].sum().reset_index()

flg['mortalityRate'] = round((flg['deaths']/flg['confirmed'])*100, 2)
temp = flg[flg['confirmed']>100]
temp = temp.sort_values('mortalityRate', ascending=False)

fig = px.bar(temp.sort_values(by="mortalityRate", ascending=False)[:10][::-1],
             x = 'mortalityRate', y = 'country', 
             title='Deaths per 100 Confirmed Cases', text='mortalityRate', height=800, orientation='h',
             color_discrete_sequence=['darkred']
            )
fig.show()

* Belgium with the most brutal of them all. Almost 16 people for every 100 that get affected die!

In [ ]:
print("Countries with Lowest Mortality Rates")
temp = flg[flg['confirmed']>100]
temp = temp.sort_values('mortalityRate', ascending=True)[['country', 'confirmed','deaths']][:16]
temp.sort_values('confirmed', ascending=False)[['country', 'confirmed','deaths']][:20].style.background_gradient(cmap='Greens')

In [ ]:
flg['recoveryRate'] = round((flg['recovered']/flg['confirmed'])*100, 2)
temp = flg[flg['confirmed']>100]
temp = temp.sort_values('recoveryRate', ascending=False)

fig = px.bar(temp.sort_values(by="recoveryRate", ascending=False)[:10][::-1],
             x = 'recoveryRate', y = 'country', 
             title='Recoveries per 100 Confirmed Cases', text='recoveryRate', height=800, orientation='h',
             color_discrete_sequence=['#2ca02c']
            )
fig.show()

In [ ]:
print("Countries with Worst Recovery Rates")
temp = flg[flg['confirmed']>100]
temp = temp.sort_values('recoveryRate', ascending=True)[['country', 'confirmed','recovered']][:20]
temp.sort_values('confirmed', ascending=False)[['country', 'confirmed','recovered']][:20].style.background_gradient(cmap='Reds')

## 6- Analysis of Pakistan's Situation 

In [ ]:
city_wise_data = pd.read_csv('../input/pakistan-corona-virus-citywise-data/PK COVID-19-3jun.csv')
city_wise_data.tail()

In [ ]:
city_wise_data.rename(columns={'Date': 'date', 
                     'Province':'province',
                     'City':'city',
                     'Cases': 'confirmed',
                     'Deaths':'deaths',
                     'Recovered':'recovered'
                    }, inplace=True)

In [ ]:
grouped_data = city_wise_data.groupby('city')['date', 'confirmed', 'deaths','recovered'].sum().reset_index()

In [ ]:
fig = px.bar(grouped_data.sort_values('confirmed', ascending=False)[:10][::-1], 
             x='confirmed', y='city', color_discrete_sequence=['#D63230'],
             title='Confirmed Cases So Far in Pakistan', text='confirmed', orientation='h')
fig.show()

In [ ]:
fig = px.bar(grouped_data.sort_values('recovered', ascending=False)[:10][::-1], 
             x='recovered', y='city', color_discrete_sequence=['#D63230'],
             title='Recovered So Far in Pakistan', text='recovered', orientation='h')
fig.show()

In [ ]:
fig = px.bar(grouped_data.sort_values('deaths', ascending=False)[:10][::-1], 
             x='deaths', y='city', color_discrete_sequence=['#D63230'],
             title='Deaths so far in Pakistan', text='deaths', orientation='h')
fig.show()

#### Merging population data

In [ ]:
data['date'] = pd.to_datetime(data['date'])
data['Day_num'] = preprocessing.LabelEncoder().fit_transform(data.date)

covid_pop = pop[['country','population']]

df_pop = data.merge(covid_pop, left_on = ['country'], right_on = ['country'], how = 'left')

na_countries = df_pop['country'][df_pop['population'].isnull()].unique()

df_pop = df_pop[~df_pop['country'].isin(na_countries)]

In [ ]:
df_fin_selc = df_pop[['country','date','confirmed','recovered','deaths','Day_num','population']]

## SIR Model

**N**: total population

**S(t)**: number of people susceptible on day t

**I(t)**: number of people infected on day t

**R(t)**: number of people recovered on day t
 
**D(t)** : number of people dead on day t

**β**: expected amount of people an infected person infects per day

**D**: number of days an infected person has and can spread the disease

**γ**: the proportion of infected recovering per day (γ = 1/D)

**a**: fatality rate

**p**: rate at which people die 


#### Choosing Pakistan for our model

In [ ]:
comp_df_pk = df_fin_selc[df_fin_selc['country']=='Pakistan']
comp_df_pk = comp_df_pk.reset_index(drop = True)
comp_df_pk.head()

In [ ]:
comp_df_pk['D'] = comp_df_pk['deaths']
comp_df_pk['R'] = comp_df_pk['recovered']
comp_df_pk['I'] = comp_df_pk['confirmed']- comp_df_pk['R'] - comp_df_pk['D']
comp_df_pk['S'] = comp_df_pk['population']-comp_df_pk['confirmed']
comp_df_pk.tail(3)

In [ ]:
cond1 = comp_df_pk.I >= 1
test = comp_df_pk['Day_num'][(cond1)]
test.head(1)

### Developing SIRD model

In [ ]:
def SIRD_testmodel(y,t,bta,gmma, alpha, rho):
    S, I, R, D = y
    
    dS_dt = -1*bta*I*S/N
    dI_dt = (bta*I*S/N) -  (1-alpha)*gmma*I - alpha*rho*I
    dR_dt = (1 - alpha)*gmma*I
    dD_dt = alpha*rho*I
    
    return ([dS_dt, dI_dt, dR_dt, dD_dt])

In [ ]:
#defining initial conditions

N = 1500
S00 = N - 2.0
I00 = 2.0
R00 = 0.0
D00 =0.0
beta = 1.0 
gamma = 1/9 # first recovery observed after 9 days of 1st infection
rho = 1/12
alpha = 0.25

t = np.linspace(0,30,875)

sol = odeint(SIRD_testmodel,[S00,I00,R00, D00],t,args = (beta,gamma, alpha, rho))
sol = np.array(sol)

In [ ]:
#plotting results

plt.figure(figsize=(6,4))
plt.plot(t, sol[:,0],label = "S(t)")
plt.plot(t, sol[:,1],label = "I(t)")
plt.plot(t, sol[:,2],label = "R(t)")
plt.plot(t, sol[:,3],label = "D(t)")
plt.legend()
plt.show()

In [ ]:
PK_S = comp_df_pk['S']
PK_I = comp_df_pk['I']
PK_R = comp_df_pk['R']
PK_D = comp_df_pk['D']

In [ ]:
def SIRD(y, t, beta, gamma, alpha, rho):
    S, I, R, D = y
    dS_dt = -1*beta*I*S/N
    dI_dt = (beta*I*S/N) -  (1-alpha)*gamma*I - alpha*rho*I
    dR_dt = (1 - alpha)*gamma*I
    dD_dt = alpha*rho*I
    return dS_dt, dI_dt, dR_dt, dD_dt

In [ ]:
def fit_SIRD_odeint(x, beta, gamma, alpha, rho): # x is day num
    return integrate.odeint(SIRD, (S0, I0, R0, D0), x, args=(beta, gamma, alpha, rho))[:,1] # return I values only (ydata)

In [ ]:
first_case_index = np.array(test.head(1))[0]

xdata = comp_df_pk.Day_num

N = comp_df_pk['population'][first_case_index]
I0 = comp_df_pk['I'][first_case_index]
S0 = comp_df_pk['S'][first_case_index]
R0 = comp_df_pk['R'][first_case_index]
D0 = comp_df_pk['D'][first_case_index]
popt, pcov = optimize.curve_fit(fit_SIRD_odeint, xdata, PK_I)
fitted = fit_SIRD_odeint(xdata, *popt)

In [ ]:
from_tomorrow = len(xdata) +  21 # data of after 21 days of 2020
days_left = 365 - from_tomorrow

In [ ]:
plt.plot(xdata, PK_I, 'o')
plt.plot(xdata, fitted)
plt.title("Fit of SIR model for Pakistan infected cases")
plt.ylabel("Population infected")
plt.xlabel("Days")
plt.show()
print("Optimal parameters: beta =", popt[0], "gamma = ", popt[1], "alpha = ", popt[2], "rho = ", popt[3])

In [ ]:
D1 = comp_df_pk['D'][len(comp_df_pk)-1]
N = comp_df_pk['population'][len(comp_df_pk)-1] - D1
S1 = comp_df_pk['S'][len(comp_df_pk)-1]
I1 = comp_df_pk['I'][len(comp_df_pk)-1]
R1 = comp_df_pk['R'][len(comp_df_pk)-1]
y = S1, I1, R1, D1
x = np.linspace(0, days_left, days_left)
# Integrate the SIR equations over the time grid, x using optimal parameters.
ret = odeint(SIRD, y, x , args=(popt[0], popt[1], popt[2], popt[3]))

In [ ]:
 #remaining_days = pd.date_range(max(comp_df_pk['date']) , max(comp_df_pk['date']) + timedelta(days=days_left+1))


Predicting the cases througout the year 2020


In [ ]:
#plotting results
plt.figure(figsize=(6,4))
plt.title('Predicted number of cases for the year 2020 in remaining days')
plt.plot(x, ret[:,1],label = "Infected")
plt.legend()
plt.show()

## References and Acknowledgements
**Data:**
* [Novel Coronavirus (COVID-19) Cases, provided by JHU CSSE](https://github.com/CSSEGISandData/COVID-19)

* [COVID-19 Complete Dataset (Updated every 24hrs)](https://www.kaggle.com/imdevskp/corona-virus-report)

* [Population by country -2020](https://www.kaggle.com/tanuprabhu/population-by-country-2020)  

* [Pakistan City wise COVID-19 data](https://www.kaggle.com/zusmani/pakistan-corona-virus-citywise-data)

**Notebooks:**
* [COVID Global Forecast SIR + XGBoost](https://www.kaggle.com/kaniya/covid-global-forecast-sir-xgboost/notebook)

**Forked From and Extended the copy of:**
* [COVID-19: Digging a bit Deeper](https://www.kaggle.com/abhinand05/covid-19-digging-a-bit-deeper/notebook)